# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al, psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE
    # and then back ...
    supercell = ase_atoms(lattice, atoms, positions) * (repeat, 1, 1)
    lattice   = load_lattice(supercell)
    positions = load_positions(supercell)
    atoms = fill(Al, length(positions))

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(lattice, atoms, positions; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
ase_atoms(setup.model).write("al_supercell.png")

<img src="https://docs.dftk.org/stable/examples/al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
is_converged = DFTK.ScfConvergenceDensity(1e-4)  # Flag convergence based on density
self_consistent_field(aluminium_setup(1); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298470561809                   -0.85    5.0
  2   -8.300205053022       -2.76       -1.25    1.0
  3   -8.300435974245       -3.64       -1.89    2.4
  4   -8.300462522122       -4.58       -2.79    2.5
  5   -8.300464500966       -5.70       -3.14    3.0
  6   -8.300464581249       -7.10       -3.29    9.1
  7   -8.300464610651       -7.53       -3.43    2.2
  8   -8.300464627564       -7.77       -3.57    1.1
  9   -8.300464638495       -7.96       -3.75    4.0
 10   -8.300464640684       -8.66       -3.83    1.0
 11   -8.300464643399       -8.57       -4.09    2.2


In [4]:
self_consistent_field(aluminium_setup(2); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -16.64196652890                   -0.70    5.9
  2   -16.64598112350       -2.40       -1.14    1.0
  3   -16.67920149152       -1.48       -1.88    2.9
  4   -16.67927946335       -4.11       -2.67    3.2
  5   -16.67928599544       -5.18       -3.00    6.9
  6   -16.67928619733       -6.69       -3.32    1.5
  7   -16.67928621990       -7.65       -3.81    1.8
  8   -16.67928622393       -8.39       -4.53    2.6


In [5]:
self_consistent_field(aluminium_setup(4); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.32484653505                   -0.56    6.5
  2   -33.33238564633       -2.12       -1.00    1.0
  3   -33.33405847076       -2.78       -1.74    5.0
  4   -33.33437548582       -3.50       -2.55    2.9
  5   -33.33693541269       -2.59       -2.54   11.6
  6   -33.33693786762       -5.61       -2.60    2.6
  7   -33.33694314620       -5.28       -3.33    1.1
  8   -33.33694358137       -6.36       -3.47    4.0
  9   -33.33694388380       -6.52       -3.96    1.4
 10   -33.33694392636       -7.37       -4.82    3.2


When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); is_converged, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298528166957                   -0.85    5.1
  2   -8.300252601458       -2.76       -1.59    1.2
  3   -8.300398502222       -3.84       -2.37    2.9
  4   -8.300327771319   +   -4.15       -2.18    4.2
  5   -8.300463671986       -3.87       -3.40    1.0
  6   -8.300464515325       -6.07       -3.68    5.1
  7   -8.300464632499       -6.93       -4.11    2.0


In [7]:
self_consistent_field(aluminium_setup(4); is_converged, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.31898433154                   -0.56    6.5
  2   -33.27964121362   +   -1.41       -1.25    1.1
  3   +7.861582680346   +    1.61       -0.26    6.5
  4   -33.30460206396        1.61       -1.51    5.8
  5   -33.19081647529   +   -0.94       -1.34    3.5
  6   -29.59382727159   +    0.56       -0.78    4.4
  7   -33.32165953382        0.57       -1.92    4.6
  8   -33.33314513388       -1.94       -1.94    2.5
  9   -33.33267359888   +   -3.33       -1.98    2.4
 10   -33.33598915570       -2.48       -2.52    2.0
 11   -33.33660711131       -3.21       -2.73    2.0
 12   -33.33669398689       -4.06       -2.84    1.8
 13   -33.33693783031       -3.61       -3.29    2.9
 14   -33.33693713230   +   -6.16       -3.56    2.8
 15   -33.33694360272       -5.19       -4.12    2.8


For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.